In [1]:
!pip install geopy unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 16.6 MB/s eta 0:00:00


In [5]:
import pandas as pd
import os
import re
import time
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import unidecode

Merging the LLM locations and rules locations

In [7]:
def agg_pages(series):
    return ','.join(sorted({str(x) for x in series if pd.notna(x)}))

def agg_mentions(series):
    return " || ".join(sorted({str(x).strip() for x in series if pd.notna(x)}))

def get_base_key(fname):
    # for match
    return (fname.lower()
            .replace('_locations.csv', '')
            .replace('_locations_llm_clean.csv', '')
            .replace('.csv', '')
            .replace(' ', '')
           )

In [9]:
def merge_rule_llm_pair(rules_csv, llm_csv, out_csv):
    import pandas as pd

    # RULES
    df_rules = pd.read_csv(rules_csv)
    if not {'filename', 'page', 'mention', 'location'}.issubset(df_rules.columns):
        print(f"Skipping {rules_csv}: missing required columns")
        return
    # Label as rules
    df_rules = df_rules[['filename', 'page', 'mention', 'location']].copy()
    df_rules["source"] = "rules"

    # LLM
    df_llm = pd.read_csv(llm_csv)
    if not {'filename', 'page', 'mention', 'location'}.issubset(df_llm.columns):
        print(f"Skipping {llm_csv}: missing required columns")
        return
    df_llm = df_llm[['filename', 'page', 'mention', 'location']].copy()
    df_llm["source"] = "llm"

    # Combine
    df_all = pd.concat([df_rules, df_llm], ignore_index=True)

    # Group and Merge Sources
    df_merged = (
        df_all
        .groupby(['filename', 'location'])
        .agg({
            'page': agg_pages,
            'mention': agg_mentions,
            'source': lambda s: ','.join(sorted(set(s))),
        })
        .reset_index()
        .rename(columns={'page': 'pages', 'mention': 'mentions'})
    )


    df_merged.to_csv(out_csv, index=False)
    print(f"Saved merged file: {out_csv}")
    return df_merged

In [11]:
def merge_all_rule_llm_files(rules_folder, llm_folder, out_folder):
    os.makedirs(out_folder, exist_ok=True)
    rules_files = {get_base_key(f): f for f in os.listdir(rules_folder) if f.endswith("_locations.csv")}
    llm_files = {get_base_key(f): f for f in os.listdir(llm_folder) if "_llm_clean.csv" in f.lower()}
    matched = rules_files.keys() & llm_files.keys()
    print(f"RULES base keys: {list(rules_files.keys())}")
    print(f"LLM base keys: {list(llm_files.keys())}")
    print(f"Found {len(matched)} matching base keys to merge.")
    for key in matched:
        rules_path = os.path.join(rules_folder, rules_files[key])
        llm_path = os.path.join(llm_folder, llm_files[key])
        out_path = os.path.join(out_folder, f"{key}_merged.csv")
        merge_rule_llm_pair(rules_path, llm_path, out_path)

In [13]:
merge_all_rule_llm_files("DATA_FOR_MODELS/Data4_rules_csv", "DATA_FOR_MODELS/Data_LLM_CSV_clean", "DATA_FOR_MODELS/Data4_merged2")

RULES base keys: ['2007_tshibubudze_themarkoyefault_2007', '2008_matabane_fe3', '2009_bontlenkuna_0605886p_honoursreport', '2010_matsheka_irvinfinalthesis', '2010_mohale_gisinterpretationofneburkinafaso', '2011_peters_eastmarkoye_2011', '2011_woolfe_thestratigraphyandmetamorphicfaciesofthekemb', '2012_simoko_petrology,geochemistryandstructureofthepissilabatholithandthesaabazonegneiss', '2013_funyufunyu', '2013_peters', '2013_ramabulana_sadiolahillpetrology', '2014_msc_yossi', '2015_lebrun_siguiri', '2015_masurel_phd']
LLM base keys: ['2007_tshibubudze_themarkoyefault_2007', '2008_matabane_fe3', '2009_bontlenkuna_0605886p_honoursreport', '2010_matsheka_irvinfinalthesis', '2010_mohale_gisinterpretationofneburkinafaso', '2011_peters_eastmarkoye_2011', '2011_woolfe_thestratigraphyandmetamorphicfaciesofthekemb', '2012_simoko_petrology,geochemistryandstructureofthepissilabatholithandthesaabazonegneiss', '2013_funyufunyu', '2013_peters', '2013_ramabulana_sadiolahillpetrology', '2014_msc_yossi

Cleaning the merged files:

Removing authors due to in text citation

Removing generic location names e.g. study area, basin

Categorizing location by type: GPS,PLACE and Approximate


In [15]:
def is_citation(mention, loc):
    # Looks for patterns like "Ledru et al.", "(Taylor, 1990)", etc.
    return bool(re.search(rf"\b{re.escape(loc)}\b.*et al\.?", mention, re.IGNORECASE)) or \
           bool(re.search(rf"\({re.escape(loc)}, \d{{4}}\)", mention))

In [17]:
import spacy
nlp = spacy.load("en_core_web_sm")
def is_person_entity(loc):
    doc = nlp(loc)
    return any(ent.label_ == "PERSON" for ent in doc.ents)

In [19]:
def is_author_like(loc, mention):
    loc_clean = loc.strip()
    if is_citation(mention, loc_clean):
        return True
    if is_person_entity(loc_clean):
        return True
    return False

In [21]:
def filter_out_authors(df):
    # Expects 'location' and 'mentions' columns
    mask = df.apply(lambda row: not is_author_like(row['location'], row['mentions']), axis=1)
    return df[mask].reset_index(drop=True)

In [23]:
GENERIC_LOCATION_WORDS = [
    "study area", "area of study", "area of investigation", "study region", "study site",
    "study location", "area", "region", "zone", "locality", "localities",
    "the area", "the region", "investigation area", "site", "sites"
]

def remove_generic_location_words(location):
    """
    Removes generic location words/phrases (case-insensitive) from start or end of a location string.
    """
    if not isinstance(location, str):
        return location
    s = location.strip()
    # Build a regex to match generic words at start or end (with optional spaces/punctuation)
    pattern = r"^(%s)\b[\s,:\-]*|[\s,:\-]*(%s)$" % (
        "|".join(map(re.escape, GENERIC_LOCATION_WORDS)),
        "|".join(map(re.escape, GENERIC_LOCATION_WORDS))
    )
    s = re.sub(pattern, "", s, flags=re.IGNORECASE)
    # Remove double spaces, commas, etc.
    s = re.sub(r"\s+", " ", s)
    s = s.strip(",.:-; ")
    return s


In [25]:
def is_gps(s):
    # Simple patterns for decimal/sexagesimal degrees or UTM
    s = str(s).strip()
    # Simple decimal degree/UTM/Easting-Northing patterns:
    if re.match(r"^\d{5,}[-; ]\s*\d{5,}$", s):  # e.g., 0184836-1587581 or 1534836; 0216256
        return True
    # Matches "N 14 36 37 8 E 00 00 12 1" etc.
    if re.search(r"[NSEW]\s*\d+", s) and re.search(r"[EW]\s*\d+", s):
        return True
    # Matches "GPS" in string (sometimes extracted as "GPS 14 35 05 7 W 00 00 05 4")
    if s.upper().startswith("GPS"):
        return True
    return False


def is_approximate(loc):
    if pd.isna(loc): return False
    loc = str(loc).strip()
    # Match "NE of Burkina Faso", "southern part of Ghana", etc.
    direction = r"(?:N|S|E|W|NE|NW|SE|SW|North|South|East|West|Northern|Southern|Eastern|Western)"
    if re.match(rf"^{direction}(\s+part)?\s+of\s+.+", loc, flags=re.IGNORECASE):
        return True
    if re.match(rf"^.+\b({direction})\b", loc, flags=re.IGNORECASE):
        return True
    return False

def is_geounit(s):
    geo_unit_keywords = [
        "belt", "craton", "basin", "shear zone", "fault", "batholith", "terrane", "inlier",
        "province", "region", "mine", "group", "supergroup", "pluton", "complex", "gneiss", "channel", "graben","domain"
    ]
    s = s.lower().strip()
    # Don't count plain country names as geounit
    return any(kw in s for kw in geo_unit_keywords)

def classify_location_type(loc):
    loc = str(loc)
    if is_gps(loc):
        return "GPS"
    elif is_approximate(loc):
        return "APPROXIMATE"
    elif is_geounit(loc):
        return "GEOLOGICAL_UNIT"
    elif len(loc) < 3:
        return "UNGEOCODED"
    else:
        return "PLACE"


While geocoding the approximate locations, the base area is extracted and treated as the main location but the geocode will indicate it will be approximate location

In [28]:
def extract_base_place(location):
    """
    Attempt to extract the most likely base location from a regional phrase.
    - Strips out directional modifiers and keeps the likely toponyms.
    - Works best for phrases like "eastern part of X", "near Y", etc.
    """
    # 1. Try 'of ...' or 'in ...'
    match = re.search(r'of ([A-Za-z \-\’\'éèêàôîïç]+)$', location)
    if match:
        return match.group(1).strip()
    match = re.search(r'in ([A-Za-z \-\’\'éèêàôîïç]+)$', location)
    if match:
        return match.group(1).strip()
    # 2. Remove phrases like 'part of', 'region of', 'border with'
    cleaned = re.sub(r'^(north|south|east|west|northern|southern|eastern|western|central|upper|lower|border|area|part|region|near|around|about|along)\s+of\s+', '', location, flags=re.I)
    cleaned = re.sub(r'^(north|south|east|west|northern|southern|eastern|western|central|upper|lower|border|area|part|region|near|around|about|along)\s+', '', cleaned, flags=re.I)
    cleaned = cleaned.strip(",.;:()[] ")

    # 3. If there are coordinates, skip
    if re.search(r'\d{4,}', cleaned):
        return location.strip()

    # 4. Return the last capitalized chunk (if any)
    matches = re.findall(r'\b([A-Z][a-zA-Z\'\-éèêàôîïç]+(?: [A-Z][a-zA-Z\'\-éèêàôîïç]+)*)\b', cleaned)
    if matches:
        return matches[-1]

    return cleaned.strip()

Geocoding using GeoPy geocoders

Output will add on: Latitude, Longitude, Location address and geocode type(approximate)

If a location is not found then no-geocode result will be output

In [31]:
geolocator = Nominatim(user_agent="geo_coding_example")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)  # Respect OSM's rate limits!

# Geocode function with handling for failures
def geopy_geocode(loc):
    if not loc or loc.lower() in {"gps", "na", "no-geocode-result"}:
        return pd.Series(["no-geocode-result"] * 4)
    # Try full location first
    try:
        result = geocode(loc, language='en', addressdetails=True, timeout=10)
        if result:
            return pd.Series([result.latitude, result.longitude, result.address, "APPROXIMATE"])
    except Exception as e:
        print(f"Error geocoding '{loc}': {e}")
        # You can sleep here for rate limits if needed
        time.sleep(1)

    # Try base place extraction as fallback
    base = extract_base_place(loc)
    if base and base.lower() != loc.lower():
        try:
            result = geocode(base, language='en', addressdetails=True, timeout=10)
            if result:
                return pd.Series([result.latitude, result.longitude, result.address, "APPROXIMATE"])
        except Exception as e:
            print(f"Error geocoding fallback '{base}': {e}")
            time.sleep(1)

    return pd.Series(["no-geocode-result"] * 4)



In [67]:
import os
import re
import pandas as pd

def is_valid_location(loc):
    """Reject citations, years, and empty strings."""
    if not isinstance(loc, str) or not loc.strip():
        return False
    if "et al" in loc.lower():
        return False
    if re.search(r"\b(19|20)\d{2}\b", loc):  # any year like 1998, 2006
        return False
    return True

def filter_out_authors(df):
    """Filter rows that look like author names instead of real places."""
    denylist = {"potrel","björklund","eriksson","milési","villeneuve",
                "attoh","ekwueme","shanmugan","sultan"}
    mask = df['location'].apply(
        lambda x: is_valid_location(x) and x.lower() not in denylist
    )
    return df[mask].reset_index(drop=True)

def clean_location_csv_folder(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for fname in os.listdir(input_folder):
        if not fname.lower().endswith(".csv"):
            continue
        print(f"Processing: {fname}")
        df = pd.read_csv(os.path.join(input_folder, fname))
        original_len = len(df)

        # Clean location column
        if 'location' in df.columns:
            df['location'] = df['location'].map(remove_generic_location_words)
        else:
            print(f"Skipping {fname}: no 'location' column")
            continue

        # Drop empty after cleaning
        df_cleaned = df[df['location'].astype(str).str.strip().astype(bool)].reset_index(drop=True)

        # Remove authors (denylist + regex)
        if {'location', 'mentions'}.issubset(df_cleaned.columns):
            df_cleaned = filter_out_authors(df_cleaned)

        # Classify location type
        if 'location' in df_cleaned.columns:
            df_cleaned['location_type'] = df_cleaned['location'].map(classify_location_type)
        else:
            df_cleaned['location_type'] = 'UNGEOCODED'

        print(f"Kept {len(df_cleaned)}/{original_len} locations after filtering.")
        # Save
        outpath = os.path.join(output_folder, fname.replace(".csv", "_final.csv"))
        df_cleaned.to_csv(outpath, index=False)
        print(f"Saved cleaned: {outpath}")

In [69]:
clean_location_csv_folder("DATA_FOR_MODELS/Data4_merged2", "DATA_FOR_MODELS/Data_merged_clean")

Processing: 2007_tshibubudze_themarkoyefault_2007_merged.csv
Kept 104/105 locations after filtering.
Saved cleaned: DATA_FOR_MODELS/Data_merged_clean\2007_tshibubudze_themarkoyefault_2007_merged_final.csv
Processing: 2008_matabane_fe3_merged.csv
Kept 146/146 locations after filtering.
Saved cleaned: DATA_FOR_MODELS/Data_merged_clean\2008_matabane_fe3_merged_final.csv
Processing: 2009_bontlenkuna_0605886p_honoursreport_merged.csv
Kept 86/89 locations after filtering.
Saved cleaned: DATA_FOR_MODELS/Data_merged_clean\2009_bontlenkuna_0605886p_honoursreport_merged_final.csv
Processing: 2010_matsheka_irvinfinalthesis_merged.csv
Kept 104/104 locations after filtering.
Saved cleaned: DATA_FOR_MODELS/Data_merged_clean\2010_matsheka_irvinfinalthesis_merged_final.csv
Processing: 2010_mohale_gisinterpretationofneburkinafaso_merged.csv
Kept 93/94 locations after filtering.
Saved cleaned: DATA_FOR_MODELS/Data_merged_clean\2010_mohale_gisinterpretationofneburkinafaso_merged_final.csv
Processing: 201

In [71]:
def geocode_location_csv_folder(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for fname in os.listdir(input_folder):
        if not fname.lower().endswith(".csv"):
            continue
        print(f"Geocoding: {fname}")
        df = pd.read_csv(os.path.join(input_folder, fname))
        if 'location' not in df.columns:
            print(f"SKIP (no location column): {fname}")
            continue
        # Clean generic location words (optional)
        df['location'] = df['location'].map(remove_generic_location_words)
        # Apply geopy_geocode, safe handling!
        geo_df = df['location'].apply(lambda loc: pd.Series(geopy_geocode(loc)))
        geo_df.columns = ['geocode_lat', 'geocode_lon', 'geocode_str', 'geocode_type']
        df = pd.concat([df, geo_df], axis=1)
        out_csv = os.path.join(output_folder, fname.replace(".csv", "_geocoded.csv"))
        df.to_csv(out_csv, index=False)
        print(f"Saved geocoded file: {out_csv}")


geocode_location_csv_folder("DATA_FOR_MODELS/Data_merged_clean", "DATA_FOR_MODELS/Data_geocoded")

Geocoding: 2007_tshibubudze_themarkoyefault_2007_merged_final.csv
Saved geocoded file: DATA_FOR_MODELS/Data_geocoded\2007_tshibubudze_themarkoyefault_2007_merged_final_geocoded.csv
Geocoding: 2008_matabane_fe3_merged_final.csv
Saved geocoded file: DATA_FOR_MODELS/Data_geocoded\2008_matabane_fe3_merged_final_geocoded.csv
Geocoding: 2009_bontlenkuna_0605886p_honoursreport_merged_final.csv
Saved geocoded file: DATA_FOR_MODELS/Data_geocoded\2009_bontlenkuna_0605886p_honoursreport_merged_final_geocoded.csv
Geocoding: 2010_matsheka_irvinfinalthesis_merged_final.csv
Saved geocoded file: DATA_FOR_MODELS/Data_geocoded\2010_matsheka_irvinfinalthesis_merged_final_geocoded.csv
Geocoding: 2010_mohale_gisinterpretationofneburkinafaso_merged_final.csv
Saved geocoded file: DATA_FOR_MODELS/Data_geocoded\2010_mohale_gisinterpretationofneburkinafaso_merged_final_geocoded.csv
Geocoding: 2011_peters_eastmarkoye_2011_merged_final.csv
Saved geocoded file: DATA_FOR_MODELS/Data_geocoded\2011_peters_eastmarkoye

In [73]:
import pandas as pd
import folium
import itertools
from pathlib import Path

# --- Paths ---
input_dir = Path("DATA_FOR_MODELS/Data_geocoded")
output_dir = Path("DATA_FOR_MODELS/Data_Maps")
output_dir.mkdir(parents=True, exist_ok=True)

# --- Function to build a map for one file ---
def build_map(csv_path, output_dir):
    df = pd.read_csv(csv_path)

    # keep only valid coords
    df = df[df['geocode_lat'].apply(lambda x: str(x).replace('.', '', 1).isdigit())]
    df['geocode_lat'] = df['geocode_lat'].astype(float)
    df['geocode_lon'] = df['geocode_lon'].astype(float)

    if df.empty:
        print(f" Skipping {csv_path.name} (no valid coordinates)")
        return

    # --- accuracy: exact or approximate ---
    def accuracy(row):
        gstr = str(row.get("geocode_str", "")).lower()
        if "approx" in gstr or "near" in gstr:
            return "approximate"
        return "exact"

    df["accuracy"] = df.apply(accuracy, axis=1)

    # --- assign colors automatically per unique location_type ---
    unique_types = df["location_type"].dropna().unique()
    color_cycle = itertools.cycle(
        ["blue","green","red","purple","orange","cyan","pink","brown","gray","olive"]
    )
    COLOR_MAP = {lt: next(color_cycle) for lt in unique_types}

    def color_for(row):
        return COLOR_MAP.get(row.get("location_type", ""), "black")

    # --- center map ---
    center_lat = df["geocode_lat"].mean()
    center_lon = df["geocode_lon"].mean()
    m = folium.Map(location=[center_lat, center_lon], zoom_start=6)

    # --- plot circles ---
    for _, row in df.iterrows():
        popup_text = (
            f"<b>Location:</b> {row.get('location_clean', '')}<br>"
            f"<b>Context:</b> {row.get('mentions', row.get('mention', ''))}<br>"
            f"<b>Geocode:</b> {row.get('geocode_str', '')}<br>"
            f"<b>Pages:</b> {row.get('pages', '')}<br>"
            f"<b>Type:</b> {row.get('location_type', '')}<br>"
            f"<b>Accuracy:</b> {row['accuracy']}"
        )

        folium.CircleMarker(
            location=[row["geocode_lat"], row["geocode_lon"]],
            radius=7,
            color=color_for(row),
            fill=True,
            fill_color=color_for(row),
            fill_opacity=0.7 if row["accuracy"] == "exact" else 0.3,
            tooltip=row.get("location_clean", ""),
            popup=folium.Popup(popup_text, max_width=350),
        ).add_to(m)

    # --- build legend dynamically ---
    legend_html = """
    <div style="position: fixed; bottom: 20px; left: 20px; z-index: 9999;
                background: white; padding: 8px 10px; border: 1px solid #ccc; border-radius: 4px;">
      <div style="font-weight: 600; margin-bottom: 6px;">Legend (location_type)</div>
    """
    for lt, col in COLOR_MAP.items():
        legend_html += f"""
          <div><span style="display:inline-block;width:12px;height:12px;background:{col};
                             margin-right:6px;border-radius:50%;"></span>{lt}</div>
        """
    legend_html += """
      <div style="margin-top:6px;"><i>Transparency: solid = exact, faint = approximate</i></div>
    </div>
    """
    m.get_root().html.add_child(folium.Element(legend_html))

    # --- save map ---
    out_path = output_dir / (csv_path.stem + "_map.html")
    m.save(out_path)
    print(f"Saved map: {out_path}")

# --- Process all CSV files ---
for csv_file in input_dir.glob("*.csv"):
    build_map(csv_file, output_dir)


Saved map: DATA_FOR_MODELS\Data_Maps\2007_tshibubudze_themarkoyefault_2007_merged_final_geocoded_map.html
Saved map: DATA_FOR_MODELS\Data_Maps\2008_matabane_fe3_merged_final_geocoded_map.html
Saved map: DATA_FOR_MODELS\Data_Maps\2009_bontlenkuna_0605886p_honoursreport_merged_final_geocoded_map.html
Saved map: DATA_FOR_MODELS\Data_Maps\2010_matsheka_irvinfinalthesis_merged_final_geocoded_map.html
Saved map: DATA_FOR_MODELS\Data_Maps\2010_mohale_gisinterpretationofneburkinafaso_merged_final_geocoded_map.html
Saved map: DATA_FOR_MODELS\Data_Maps\2011_peters_eastmarkoye_2011_merged_final_geocoded_map.html
Saved map: DATA_FOR_MODELS\Data_Maps\2011_woolfe_thestratigraphyandmetamorphicfaciesofthekemb_merged_final_geocoded_map.html
Saved map: DATA_FOR_MODELS\Data_Maps\2012_simoko_petrology,geochemistryandstructureofthepissilabatholithandthesaabazonegneiss_merged_final_geocoded_map.html
Saved map: DATA_FOR_MODELS\Data_Maps\2013_funyufunyu_merged_final_geocoded_map.html
Saved map: DATA_FOR_MODEL